# Construction de la base de données

Ce notebook documente la construction de la base finale au niveau *musée–année* à partir de trois sources :
- Museofile (caractéristiques des musées),
- Entrées et catégories de public (fréquentation annuelle et composition du public),
- Fréquentation historique (Excel 2001–2016) utilisée comme source complémentaire.


In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd

# Assure l'import de src/ depuis notebook/
ROOT = Path.cwd().parents[0] # Exécution depuis le dossier notebook/
sys.path.append(str(ROOT))

from src.build_bases import (
    load_raw_data,
    build_dim_musees,
    build_fact_frequentation,
    build_fact_freq_excel,
    merge_dataset,
    basic_quality_checks,
)
from src.cleaning import clean_and_enrich
from src.chemins import OUTPUT_DIR

## 1) Chargement des données brutes

On charge les trois fichiers bruts depuis le dossier `data/` via la fonction `load_raw_data()`.
On vérifie rapidement les dimensions et quelques colonnes clés.


In [2]:
freq_raw, ent_raw, museo_raw = load_raw_data()

display(freq_raw.head(3))
display(ent_raw.head(3))
display(museo_raw.head(3))

print("freq_raw :", freq_raw.shape)
print("ent_raw  :", ent_raw.shape)
print("museo_raw:", museo_raw.shape)

Chargement des données brutes...
  freq_raw shape  : (1241, 22)
  ent_raw shape   : (11252, 20)
  museo_raw shape : (1222, 24)


,REF DU MUSEE,NEW REGIONS,NOM DU MUSEE,VILLE,Fréquentation,2001,2002,2003,2004,2005,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 21
0,0105301,AUVERGNE-RHÔNE-ALPES,Musée du Brou,BOURG-EN-BRESSE,Totale,74056,80303,70599,71456,68960,...,60486,69815,74028,79095,74222,75725,94648,112860,98951,NaN
1,0105302,AUVERGNE-RHÔNE-ALPES,Musée Départemental des Pays De l'Ain,BOURG-EN-BRESSE,Totale,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0106401,AUVERGNE-RHÔNE-ALPES,Musée de la Société d'Histoire et d'Archéologie,BRIORD,Totale,100,147,137,179,35,...,NC,161,308,NC,NC,NC,NC,NC,NC,NaN


,IDPatrimostat,IDMuseofile,region,departement,dateappellation,ferme,anneefermeture,nom_du_musee,lien_avec,ville,codeInseeCommune,annee,payant,gratuit,total,individuel,scolaires,groupes_hors_scolaires,moins_18_ans_hors_scolaires,_18_25_ans
0,0105301,M0963,Auvergne-Rhône-Alpes,Ain,01/02/2003,NON,NaN,musée de Brou,NaN,BOURG EN BRESSE,01053,2014,53088.0,41560.0,94648.0,NaN,NaN,NaN,5186.0,2157.0
1,0105301,M0963,Auvergne-Rhône-Alpes,Ain,01/02/2003,NON,NaN,musée de Brou,NaN,BOURG EN BRESSE,01053,2015,76918.0,35942.0,112860.0,NaN,NaN,NaN,NaN,NaN
2,0105301,M0963,Auvergne-Rhône-Alpes,Ain,01/02/2003,NON,NaN,musée de Brou,NaN,BOURG EN BRESSE,01053,2016,74138.0,24813.0,98951.0,NaN,NaN,NaN,NaN,NaN


,Identifiant,Nom_officiel,Adresse,Lieu,Code_postal,Ville,Departement,Region,URL,Telephone,...,Themes,Artiste,Personnage_phare,Interet,Protection_batiment,Protection_espace,Refmer,Annee_creation,Date_de_mise_a_jour,Coordonnees
0,M1128,musée des sapeurs-pompiers de Lyon,358 avenue de Champagne,Centre de secours principal,69009,Lyon,Rhône,Auvergne-Rhône-Alpes,museepompiers.com/,04 72 17 54 54,...,"Armes, Uniformes, Véhicules pompiers, Archives...",NaN,NaN,NaN,NaN,NaN,SPR8400131,1971,2025-10-13,"45.790491, 4.797411"
1,M0001,musée de la Folie Marco,30 rue du docteur Sultzer,NaN,67140,Barr,Bas-Rhin,Grand Est,www.musee-foliemarco.com,03 88 08 94 72,...,"Arts décoratifs : Mobilier, Objets d'art",NaN,NaN,Demeure patricienne d'architecture française d...,Protégé au titre des Monuments historiques : h...,Abords d'un Monument historique. Site inscrit.,PA00084599;IA00115077,1960,2025-10-13,"48.410166, 7.451102"
2,M0002,musée de la poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Bas-Rhin,Grand Est,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,...,Arts décoratifs : Céramique,NaN,NaN,Maison à pans de bois du début du XVIIIe siècle.,NaN,NaN,NaN,1981,2025-10-13,"48.900348, 7.914409"


freq_raw : (1241, 22)
ent_raw  : (11252, 20)
museo_raw: (1222, 24)


## 2) Construction des tables intermédiaires

On construit :
- `musees` : table de dimension (caractéristiques fixes des musées)
- `frequentation_annuelle` : table de faits (musée–année, fréquentation + composition du public)
- `freq_excel_long` : fréquentation historique au format long (musée–année)


In [ ]:
musees = build_dim_musees(museo_raw)
frequentation_annuelle = build_fact_frequentation(ent_raw)
freq_excel_long = build_fact_freq_excel(freq_raw)

print("musees :", musees.shape)
print("frequentation_annuelle :", frequentation_annuelle.shape)
print("freq_excel_long :", freq_excel_long.shape)

display(musees.head(3))
display(frequentation_annuelle.head(3))
display(freq_excel_long.head(3))


Aperçu fact_frequentation :
  id_patrimostat id_museofile  annee     total   payant  gratuit
0        0105301        M0963   2014   94648.0  53088.0  41560.0
1        0105301        M0963   2015  112860.0  76918.0  35942.0
2        0105301        M0963   2016   98951.0  74138.0  24813.0
3        0105301        M0963   2017   97132.0  73396.0  23736.0
4        0105301        M0963   2018   98216.0  62284.0  35932.0

Aperçu fact_freq_excel :
  id_patrimostat  annee  total_frequentation
0        0105301   2001              74056.0
1        0105302   2001                  NaN
2        0106401   2001                100.0
3        0113701   2001              22234.0
4        0119201   2001                204.0
musees : (1222, 20)
frequentation_annuelle : (11250, 22)
freq_excel_long : (19856, 7)


,id_museofile,nom_officiel,Adresse,Lieu,Code_postal,ville,departement,region,URL,Telephone,categorie,domaine_thematique,Protection_batiment,Protection_espace,Refmer,annee_creation,Date_de_mise_a_jour,coordonnees,latitude,longitude
0,M1128,musée des sapeurs-pompiers de Lyon,358 avenue de Champagne,Centre de secours principal,69009,Lyon,Rhône,Auvergne-Rhône-Alpes,museepompiers.com/,04 72 17 54 54,NaN,Ethnologie;Histoire;Technique et industrie,NaN,NaN,SPR8400131,1971,2025-10-13,"45.790491, 4.797411",45.790491,4.797411
1,M0001,musée de la Folie Marco,30 rue du docteur Sultzer,NaN,67140,Barr,Bas-Rhin,Grand Est,www.musee-foliemarco.com,03 88 08 94 72,Musée de site,Arts décoratifs,Protégé au titre des Monuments historiques : h...,Abords d'un Monument historique. Site inscrit.,PA00084599;IA00115077,1960,2025-10-13,"48.410166, 7.451102",48.410166,7.451102
2,M0002,musée de la poterie,2 rue de Kuhlendorf,NaN,67660,Betschdorf,Bas-Rhin,Grand Est,www.betschdorf.com/vie-culturelle-et-associati...,03 88 54 48 07,musée en milieu rural,Arts décoratifs;Technique et industrie;Ethnologie,NaN,NaN,NaN,1981,2025-10-13,"48.900348, 7.914409",48.900348,7.914409


,id_patrimostat,id_museofile,region,departement,dateappellation,ferme,anneefermeture,nom_du_musee,ville,codeInseeCommune,...,gratuit,total,individuel,scolaires,groupes_hors_scolaires,moins_18_ans_hors_scolaires,_18_25_ans,part_gratuit,part_scolaires,part_individuels
0,0105301,M0963,Auvergne-Rhône-Alpes,Ain,01/02/2003,NON,NaN,musée de Brou,BOURG EN BRESSE,01053,...,41560.0,94648.0,NaN,NaN,NaN,5186.0,2157.0,0.439101,NaN,NaN
1,0105301,M0963,Auvergne-Rhône-Alpes,Ain,01/02/2003,NON,NaN,musée de Brou,BOURG EN BRESSE,01053,...,35942.0,112860.0,NaN,NaN,NaN,NaN,NaN,0.318465,NaN,NaN
2,0105301,M0963,Auvergne-Rhône-Alpes,Ain,01/02/2003,NON,NaN,musée de Brou,BOURG EN BRESSE,01053,...,24813.0,98951.0,NaN,NaN,NaN,NaN,NaN,0.250760,NaN,NaN


,id_patrimostat,region_excel,nom_musee_excel,ville_excel,type_frequentation,annee,total_frequentation
0,0105301,AUVERGNE-RHÔNE-ALPES,Musée du Brou,BOURG-EN-BRESSE,Totale,2001,74056.0
1,0105302,AUVERGNE-RHÔNE-ALPES,Musée Départemental des Pays De l'Ain,BOURG-EN-BRESSE,Totale,2001,NaN
2,0106401,AUVERGNE-RHÔNE-ALPES,Musée de la Société d'Histoire et d'Archéologie,BRIORD,Totale,2001,100.0


## 3) Fusion des tables : création de `df_modele`

La base finale est construite au niveau *musée–année*.  
On fusionne la table de fréquentation annuelle avec les caractéristiques des musées, puis on ajoute
la fréquentation historique lorsqu’elle est disponible.


In [ ]:
df_modele = merge_dataset(musees, frequentation_annuelle, freq_excel_long)
print("df_modele :", df_modele.shape)
display(df_modele.head(5))


Aperçu df_modele (fusion) :
  id_patrimostat id_museofile  annee     total  total_frequentation  \
0        0105301        M0963   2014   94648.0              94648.0   
1        0105301        M0963   2015  112860.0             112860.0   
2        0105301        M0963   2016   98951.0              98951.0   
3        0105301        M0963   2017   97132.0                  NaN   
4        0105301        M0963   2018   98216.0                  NaN   

    nom_officiel                region  
0  musée de Brou  Auvergne-Rhône-Alpes  
1  musée de Brou  Auvergne-Rhône-Alpes  
2  musée de Brou  Auvergne-Rhône-Alpes  
3  musée de Brou  Auvergne-Rhône-Alpes  
4  musée de Brou  Auvergne-Rhône-Alpes  

Taille df_modele : (11250, 29)
df_modele : (11250, 29)


,id_patrimostat,id_museofile,dateappellation,ferme,anneefermeture,nom_du_musee,ville,codeInseeCommune,annee,payant,...,part_individuels,nom_officiel,region,departement,categorie,domaine_thematique,annee_creation,latitude,longitude,total_frequentation
0,0105301,M0963,01/02/2003,NON,NaN,musée de Brou,BOURG EN BRESSE,01053,2014,53088.0,...,NaN,musée de Brou,Auvergne-Rhône-Alpes,Ain,NaN,Archéologie;Arts décoratifs;Art moderne et con...,NaN,46.196549,5.236521,94648.0
1,0105301,M0963,01/02/2003,NON,NaN,musée de Brou,BOURG EN BRESSE,01053,2015,76918.0,...,NaN,musée de Brou,Auvergne-Rhône-Alpes,Ain,NaN,Archéologie;Arts décoratifs;Art moderne et con...,NaN,46.196549,5.236521,112860.0
2,0105301,M0963,01/02/2003,NON,NaN,musée de Brou,BOURG EN BRESSE,01053,2016,74138.0,...,NaN,musée de Brou,Auvergne-Rhône-Alpes,Ain,NaN,Archéologie;Arts décoratifs;Art moderne et con...,NaN,46.196549,5.236521,98951.0
3,0105301,M0963,01/02/2003,NON,NaN,musée de Brou,BOURG EN BRESSE,01053,2017,73396.0,...,0.793405,musée de Brou,Auvergne-Rhône-Alpes,Ain,NaN,Archéologie;Arts décoratifs;Art moderne et con...,NaN,46.196549,5.236521,NaN
4,0105301,M0963,01/02/2003,NON,NaN,musée de Brou,BOURG EN BRESSE,01053,2018,62284.0,...,0.866559,musée de Brou,Auvergne-Rhône-Alpes,Ain,NaN,Archéologie;Arts décoratifs;Art moderne et con...,NaN,46.196549,5.236521,NaN


## 4) Nettoyage et enrichissement

C'est ici que nous transformons les données brutes en variables exploitables pour la modélisation. Nous appliquons la fonction `clean_and_enrich()` définie dans `src/cleaning.py` qui effectue les opérations suivantes :

1.  **Typage des données :** Conversion robuste des colonnes numériques (gestion des erreurs de format).
2.  **Imputation simple :** Si `total_frequentation` est manquant, on utilise la colonne `total` issue de la base principale.
3.  **Variables Temporelles :**
    * Calcul de l'**âge du musée** (Année courante - Date de création).
    * Calcul des **Lags (t-1)** : Fréquentation de l'année précédente.
    * Calcul de la **Croissance** : Variation relative sécurisée (gestion des divisions par zéro).
4.  **Variables Thématiques (One-Hot Encoding) :**
    * Traitement du champ `domaine_thematique` (souvent avec plusieurs valeurs, ex: "Art;Histoire").
    * Création de **variables binaires** (`is_beaux_arts`, `is_histoire`, etc.) pour les thèmes les plus fréquents. Cela permettra au modèle de sélectionner précisément l'impact de chaque étiquette.
5.  **Variables Géographiques :**
    * Création d'une indicatrice `est_idf` (Île-de-France vs Reste).

In [9]:
# Application des transformations
df_modele_clean = clean_and_enrich(df_modele)

print("Dimensions après nettoyage :", df_modele_clean.shape)

# Vérification des Domaines (Colonnes binaires pour le modèle)
print("\n--- Vérification des variables binaires (One-Hot) ---")
cols_binaires = [c for c in df_modele_clean.columns if c.startswith("is_")]
if cols_binaires:
    print(f"Colonnes créées ({len(cols_binaires)}) : {cols_binaires[:5]} ...")
else:
    print("Aucune colonne 'is_...' trouvée.")

#Vérification des Catégories (Nettoyage texte)
print("\n--- Vérification des Catégories ---")
if "categorie" in df_modele_clean.columns:
    print("Top 10 des catégories :")
    print(df_modele_clean["categorie"].value_counts(dropna=False).head(10))

#Aperçu final
print("\n--- Aperçu des données ---")
# On retire 'domaine_simple' de la liste de visualisation puisqu'elle n'existe pas encore
cols_view = [
    "id_museofile", "annee", "total", 
    "croissance_total", 
    "categorie", "est_idf"
]
cols_final = [c for c in cols_view if c in df_modele_clean.columns]

display(df_modele_clean[cols_final].head())

Traitement des domaines thématiques...

Aperçu df_modele après nettoyage/enrichissement :
  id_museofile  annee     total  total_frequentation  age_musee  is_histoire  \
0        M0963   2014   94648.0              94648.0        NaN            1   
1        M0963   2015  112860.0             112860.0        NaN            1   
2        M0963   2016   98951.0              98951.0        NaN            1   
3        M0963   2017   97132.0                  NaN        NaN            1   
4        M0963   2018   98216.0                  NaN        NaN            1   

   is_beaux_arts  
0              1  
1              1  
2              1  
3              1  
4              1  
 Nettoyage des catégories...
 Dimensions après nettoyage : (11627, 55)

--- Vérification des variables binaires (One-Hot) ---
Colonnes créées (19) : ['is_histoire', 'is_beaux_arts', 'is_ethnologie', 'is_archéologie', 'is_arts_décoratifs'] ...

--- Vérification des Catégories ---
Top 10 des catégories :
categorie
A

,id_museofile,annee,total,croissance_total,categorie,est_idf
0,M0963,2014,94648.0,NaN,Autre,0
1,M0963,2015,112860.0,0.192418,Autre,0
2,M0963,2016,98951.0,-0.123241,Autre,0
3,M0963,2017,97132.0,-0.018383,Autre,0
4,M0963,2018,98216.0,0.011160,Autre,0


## 5) Contrôles qualité

On applique quelques contrôles  :
- cohérence des tailles (nombre de musées)
- absence de doublons musée–année
- absence de valeurs négatives aberrantes sur la fréquentation

On complète par des vérifications sur les valeurs manquantes des variables clés utilisées ensuite.


In [11]:
basic_quality_checks(musees, frequentation_annuelle, freq_excel_long, df_modele_clean)

# Vérifs manquants sur variables clés
cols_check = ["age_musee", "total_t_1", "croissance_total", "region", "latitude", "longitude"]
missing_report = (
    df_modele_clean[cols_check]
    .isna()
    .mean()
    .sort_values(ascending=False)
    .to_frame("share_missing")
)

missing_report

Nombre de musées (dim_musees)      : 1222
Nombre de musées (fact_frequent.) : 1224
Nombre de musées (df_modele)      : 1224
Années couvertes dans df_modele   : 2014–2023
Doublons (id_museofile, annee) : 496
Il y a des doublons musée/année dans df_modele.
Entrées totales négatives : 0


,share_missing
age_musee,0.481208
croissance_total,0.238841
total_t_1,0.137525
latitude,0.011869
longitude,0.011869
region,0.000000


### Contrôles qualité et diagnostics

La base finale comprend environ 1 220 musées, ce qui est cohérent entre la table de dimension
(`dim_musees`) et les tables de fréquentation. La période couverte s’étend de 2014 à 2023.

Un nombre limité de doublons au niveau musée–année est détecté (496 observations). Ces doublons
proviennent de certaines sources de fréquentation et correspondent à des déclarations multiples
pour une même année. Ils sont conservés à ce stade afin de ne pas introduire de biais arbitraire,
mais seront traités implicitement lors des analyses économétriques (via l’agrégation ou les
termes retardés).

Aucune valeur négative aberrante n’est observée pour la fréquentation totale.


### Valeurs manquantes

La base présente des valeurs manquantes pour certaines variables construites.

En particulier, l’âge du musée est manquant pour environ 48 % des observations, ce qui s’explique
par l’absence d’année de création pour de nombreux musées dans la base Museofile. De même, la
croissance annuelle et le terme retardé de fréquentation présentent des valeurs manquantes
respectivement pour environ 20 % et 11 % des observations, ce qui est attendu pour des variables
dynamiques (première année observée ou discontinuités temporelles).

Les coordonnées géographiques sont quant à elles quasi complètes (moins de 2 % de valeurs
manquantes), et la variable région est entièrement renseignée.


## 6) Export des fichiers

On exporte les tables construites dans `output/` afin de réutiliser la base dans les notebooks suivants
(stats descriptives, modélisation).


In [12]:
OUTPUT_DIR.mkdir(exist_ok=True)

musees.to_csv(OUTPUT_DIR / "musees.csv", index=False)
frequentation_annuelle.to_csv(OUTPUT_DIR / "frequentation_annuelle.csv", index=False)
freq_excel_long.to_csv(OUTPUT_DIR / "frequentation_excel_long.csv", index=False)
df_modele_clean.to_csv(OUTPUT_DIR / "df_modele_musees.csv", index=False)

print("Fichiers exportés dans :", OUTPUT_DIR.resolve())

Fichiers exportés dans : C:\Users\valen\OneDrive\Documents\GitHub\prediction-musee\output


## Conclusion

Nous obtenons une base finale structurée au niveau *musée–année*, combinant la fréquentation, la
composition du public et les caractéristiques structurelles des musées. Cette base nettoyée et enrichie
servira ensuite aux statistiques descriptives, aux visualisations géographiques et aux modèles de prédiction.


La base ainsi construite et contrôlée constitue le socle des analyses descriptives et
économétriques présentées dans la suite du projet.
